In [1]:
!pygmentize add.c
!clang-4.0 add.c -O2 -march=native -fPIC -shared -o add.so

#include <stdint.h>
#include <stdlib.h>

void add(uint32_t* res, uint32_t const* a, uint32_t const* b, size_t n) {
  for (size_t i= 0; i < n; ++i) {
    res[i] = a[i] + b[i];
  }
}


In [2]:
import pydffi
import os
pydffi.dlopen(os.path.join(os.getcwd(), "add.so"))
FFI = pydffi.FFI()
CU = FFI.cdef('''
#include <stdint.h>
#include <stdlib.h>
void add(uint32_t* res, uint32_t const* a, uint32_t const* b, size_t n);
''')
add_c = CU.funcs.add

In [4]:
import numpy as np
N = 200000
a = np.random.randint(2**32, size=N, dtype=np.uint32)
b = np.random.randint(2**32, size=N, dtype=np.uint32)

In [4]:
res = np.ndarray(N, dtype=np.uint32)
arTy = FFI.arrayType(FFI.UInt32Ty, N)
carTy = pydffi.const(arTy)
cres = pydffi.ptr(pydffi.view_as(arTy, res))
ca = pydffi.ptr(pydffi.view_as(carTy, a))
cb = pydffi.ptr(pydffi.view_as(carTy,b))
%timeit add_c(cres, ca, cb, N)

50.5 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [1]:
%reload_ext pythran.magic

In [2]:
%%pythran -O2 -march=native
#pythran export add_pythran(uint32[],uint32[])
def add_pythran(a,b):
    return a+b

/usr/bin/ld: cannot find -lcblas
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -lcblas
collect2: error: ld returned 1 exit status


In file included from /tmp/tmpm_pynkz9.cpp:2:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/core.hpp:20:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/types/combined.hpp:4:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/include/types/combined.hpp:4:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/include/types/traits.hpp:4:
In file included from /usr/lib/gcc/x86_64-linux-gnu/8/../../../../include/c++/8/complex:45:
In file included from /usr/lib/gcc/x86_64-linux-gnu/8/../../../../include/c++/8/sstream:38:
In file included from /usr/lib/gcc/x86_64-linux-gnu/8/../../../../include/c++/8/istream:38:
In file included from /usr/lib/gcc/x86_64-linux-gnu/8/../../../../include/c++/8/ios:40:
In file included from /usr/lib/gcc/x86_64-linux-gnu/8/../../.

In [5]:
%timeit add_pythran(a,b)

117 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
# Back to slides

In [8]:
!clang-4.0 -Rpass=loop-vectorize -Rpass-missed=loop-vectorize -Rpass-analysis=loop-vectorize add.c -O2 -march=native -c -o /dev/null

add.c:5:3: remark: vectorized loop
      (vectorization width: 8, interleaved count: 4) [-Rpass=loop-vectorize]
  for (size_t i= 0; i < n; ++i) {
  ^


In [9]:
!pythran -E add.py 

In [10]:
!pygmentize add.cpp

#define BOOST_SIMD_NO_STRICT_ALIASING 1
#include <pythonic/core.hpp>
#include <pythonic/python/core.hpp>
#include <pythonic/types/bool.hpp>
#include <pythonic/types/int.hpp>
#ifdef _OPENMP
#include <omp.h>
#endif
#include <pythonic/include/types/uint32.hpp>
#include <pythonic/include/types/ndarray.hpp>
#include <pythonic/types/ndarray.hpp>
#include <pythonic/types/uint32.hpp>
#include <pythonic/include/operator_/add.hpp>
#include <pythonic/operator_/add.hpp>
namespace __pythran_add
{
  struct add
  {
    typedef void callable;
    typedef void pure;
    template <typename argument_type0 , typename argument_type1 >
    struct type
    {
      typedef typename std::remove_cv<typename std::remove_reference<argument_type0>::type>::type __type0;
      typedef typename std::remove_cv<typename std::remove_reference<argument_type1>::type>::type __type1;
      typedef typename pythonic::returnable<decltype((pythonic::operator_::add(std::declval<__type0>(), std::declval<__type1>())))>::type resu

In [11]:
!clang++-4.0  add.cpp -Rpass=loop-vectorize -DNDEBUG -O2  -march=native  -fPIC -DENABLE_PYTHON_MODULE -D__PYTHRAN__=3 -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/include -I/home/aguinet/.virtualenvs/fixcompilerbug/include -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/include -I/home/aguinet/.virtualenvs/fixcompilerbug/include -I/usr/include/python3.6m -I/home/aguinet/.virtualenvs/fixcompilerbug/include/python3.6m  -std=c++11 -c -o /dev/null

add.cpp:120:17: warning: unknown attribute 'externally_visible' ignored
      [-Wunknown-attributes]
__attribute__ ((externally_visible))
                ^
1 warning generated.


In [12]:
!clang++-4.0  add.cpp -Rpass=loop-vectorize -Rpass-missed=loop-vectorize -Rpass-analysis=loop-vectorize -DNDEBUG -O2  -march=native  -fPIC -DENABLE_PYTHON_MODULE -D__PYTHRAN__=3 -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/include -I/home/aguinet/.virtualenvs/fixcompilerbug/include -I/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/numpy/core/include -I/usr/local/include -I/usr/include -I/home/aguinet/.virtualenvs/fixcompilerbug/include -I/usr/include/python3.6m -I/home/aguinet/.virtualenvs/fixcompilerbug/include/python3.6m  -std=c++11 -c -o /dev/null

add.cpp:120:17: warning: unknown attribute 'externally_visible' ignored
      [-Wunknown-attributes]
__attribute__ ((externally_visible))
                ^
In file included from add.cpp:10:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/include/types/ndarray.hpp:35:
/home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/include/types/numpy_expr.hpp:24:20: remark: 
      loop not vectorized: value that could not be identified as reduction is
      used outside the loop [-Rpass-analysis=loop-vectorize]
      return Op{}(*std::get<I>(iters)...);
                   ^
In file included from add.cpp:2:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/core.hpp:20:
In file included from /home/aguinet/.virtualenvs/fixcompilerbug/lib/python3.6/site-packages/pythran/pythonic/types/combined.hpp:4:
In file included from /home/aguinet/.virtualenvs/f

1 warning generated.


In [ ]:
# Back to slides

In [13]:
%%pythran -O2 -DUSE_BOOST_SIMD 
#pythran export add_pythran_simd(uint32[],uint32[])
def add_pythran_simd(a,b):
    return a+b

In [14]:
%timeit add_pythran_simd(a,b)

42.9 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
